In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

# =============================================================================
# [PART 1 & 2] 데이터 로드, 전처리, 지표 산출 (기존과 동일)
# =============================================================================
print("🚀 데이터 로드 및 처리 시작...")
try:
    orders = pd.read_csv('Data/olist_orders_dataset.csv')
    items = pd.read_csv('Data/olist_order_items_dataset.csv')
    products = pd.read_csv('Data/olist_products_dataset.csv')
    customers = pd.read_csv('Data/olist_customers_dataset.csv')
    reviews = pd.read_csv('Data/olist_order_reviews_dataset.csv')
    product_trans = pd.read_csv('Data/product_category_name_translation.csv')
    geo = pd.read_csv('Data/olist_geolocation_dataset.csv')
except FileNotFoundError:
    print("❌ 오류: 'Data/' 폴더에 CSV 파일이 있는지 확인해주세요.")
    exit()

geo_agg = geo.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

df = pd.merge(orders, items, on='order_id')
df = pd.merge(df, products, on='product_id')
df = pd.merge(df, customers, on='customer_id')
df = pd.merge(df, reviews, on='order_id')
df = pd.merge(df, product_trans, on='product_category_name', how='left')
df = pd.merge(df, geo_agg, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

time_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in time_cols:
    df[col] = pd.to_datetime(df[col])

df = df.dropna(subset=['geolocation_lat', 'geolocation_lng'])
df['freight_ratio'] = df['freight_value'] / df['price']
df['arrival_diff'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.days

print("✅ 마스터 테이블 생성 완료")

print("📊 지역별 전략 지표 계산 중...")
state_stats = df.groupby('customer_state').agg({
    'order_id': 'count',
    'freight_ratio': 'mean',
    'arrival_diff': 'mean',
    'review_score': 'mean',
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

state_stats['norm_demand'] = (state_stats['order_id'] - state_stats['order_id'].min()) / \
                             (state_stats['order_id'].max() - state_stats['order_id'].min())
state_stats['norm_sat'] = (state_stats['review_score'] - state_stats['review_score'].min()) / \
                          (state_stats['review_score'].max() - state_stats['review_score'].min())
state_stats['opp_score'] = state_stats['norm_demand'] * (1 - state_stats['norm_sat'])
state_stats.loc[state_stats['customer_state'] == 'RJ', 'opp_score'] *= 1.5          # 기회 점수가 압도적으로 높은 RJ에 전략적 가중치 부여


# =============================================================================
# [PART 3] 3-Layer 전략 대시보드 지도 시각화 (수정됨)
# =============================================================================
print("🗺️ 3. Folium 지도 생성 중...")

m = folium.Map(location=[-14.2350, -51.9253], zoom_start=4, tiles='CartoDB dark_matter')

# -------------------------------------------------------
# Layer 1: [Opportunity] 기회 점수 (Main Layer) - 수치 라벨 추가
# -------------------------------------------------------
fg_opportunity = folium.FeatureGroup(name='1. 기회 점수 (Opportunity)')

for _, row in state_stats.iterrows():
    score = row['opp_score']
    radius = 10 + (score * 60)
    
    if score > 0.15: color = '#ffd700'
    elif score > 0.05: color = '#9c27b0'
    else: color = '#4682b4'

    tooltip_html = f"""
    <h4>State: {row['customer_state']}</h4>
    <b>⭐ Opp Score: {score:.3f}</b><br>
    📦 Demand: {row['order_id']}<br>
    😊 Avg Rating: {row['review_score']:.2f}
    """

    # 원형 마커
    folium.CircleMarker(
        location=[row['geolocation_lat'], row['geolocation_lng']],
        radius=radius,
        color=color, fill=True, fill_color=color, fill_opacity=0.6,
        tooltip=tooltip_html
    ).add_to(fg_opportunity)
    
    # [추가] 수치 텍스트 라벨
    folium.map.Marker(
        [row['geolocation_lat'], row['geolocation_lng']],
        icon=folium.DivIcon(
            icon_size=(100, 20),
            icon_anchor=(50, 0), # 마커 중앙 하단에 위치
            html=f'<div style="font-size: 10pt; font-weight: bold; color: white; text-align: center; text-shadow: 1px 1px 2px black;">{score:.3f}</div>'
        )
    ).add_to(fg_opportunity)

fg_opportunity.add_to(m)

# -------------------------------------------------------
# Layer 2: [Risk] 배송 위험 지역 (Overlay) - 수치 라벨 추가
# -------------------------------------------------------
fg_risk = folium.FeatureGroup(name='2. 배송 위험 (Reliability)', show=False)

for _, row in state_stats.iterrows():
    if row['arrival_diff'] < 12: 
        # 아이콘 마커
        folium.Marker(
            location=[row['geolocation_lat'], row['geolocation_lng']],
            icon=folium.Icon(color='red', icon='exclamation-triangle', prefix='fa'),
            tooltip=f"<b>⚠️ Risk Alert ({row['customer_state']})</b><br>여유 기간: {row['arrival_diff']:.1f}일"
        ).add_to(fg_risk)
        
        # [추가] 수치 텍스트 라벨
        folium.map.Marker(
            [row['geolocation_lat'], row['geolocation_lng']],
            icon=folium.DivIcon(
                icon_size=(100, 20),
                icon_anchor=(0, -10), # 아이콘 오른쪽에 위치
                html=f'<div style="font-size: 9pt; font-weight: bold; color: #ffcccc; text-align: left; text-shadow: 1px 1px 1px black;">{row["arrival_diff"]:.1f}일</div>'
            )
        ).add_to(fg_risk)

fg_risk.add_to(m)

# -------------------------------------------------------
# Layer 3: [Cost] 고비용 지역 (Overlay) - 수치 라벨 추가
# -------------------------------------------------------
fg_cost = folium.FeatureGroup(name='3. 고비용 지역 (Bundling)', show=False)

for _, row in state_stats.iterrows():
    if row['freight_ratio'] > 0.4:
        # 아이콘 마커 (위치 약간 조정)
        folium.Marker(
            location=[row['geolocation_lat'] + 0.5, row['geolocation_lng'] + 0.5],
            icon=folium.Icon(color='green', icon='money', prefix='fa'),
            tooltip=f"<b>💰 Cost Alert ({row['customer_state']})</b><br>배송비 비율: {row['freight_ratio']:.2f}"
        ).add_to(fg_cost)
        
        # [추가] 수치 텍스트 라벨
        folium.map.Marker(
            [row['geolocation_lat'] + 0.5, row['geolocation_lng'] + 0.5],
            icon=folium.DivIcon(
                icon_size=(100, 20),
                icon_anchor=(0, -10), # 아이콘 오른쪽에 위치
                html=f'<div style="font-size: 9pt; font-weight: bold; color: #90ee90; text-align: left; text-shadow: 1px 1px 1px black;">{row["freight_ratio"]:.2f}</div>'
            )
        ).add_to(fg_cost)

fg_cost.add_to(m)

# -------------------------------------------------------
# 레이어 컨트롤 & 범례 추가 (범례 상자 크기 및 스타일 개선)
# -------------------------------------------------------
folium.LayerControl(collapsed=False).add_to(m)

# [수정] 범례 상자 크기(width, height) 및 스타일 개선
legend_html = '''
     <div style="position: fixed; bottom: 30px; right: 30px; width: 280px; height: 250px; 
     border:2px solid grey; z-index:9999; font-size:14px; line-height: 1.6;
     background-color:rgba(255,255,255,0.9); padding: 15px; border-radius:10px; box-shadow: 3px 3px 5px rgba(0,0,0,0.3);">
     
     <h4 style="margin-top:0; text-align:center;">📊 지표별 대시보드 </h4>
     
     <div style="margin-bottom: 10px;">
       <i style="background:#ffd700; width:15px; height:15px; display:inline-block; border-radius:50%; border:1px solid #666; vertical-align:middle;"></i> 
       <b>기회 점수 (Main)</b><br>
       <span style="font-size:12px; color:#555; margin-left: 20px;">지도상 수치 = 기회 점수</span>
     </div>

     <div style="margin-bottom: 10px;">
       <i style="background:red; width:15px; height:15px; display:inline-block; border-radius:50%; border:1px solid #666; vertical-align:middle;"></i> 
       <b>배송 위험 (Risk)</b><br>
       <span style="font-size:12px; color:#555; margin-left: 20px;">지도상 수치 = 여유 기간(일)</span>
     </div>

     <div style="margin-bottom: 0px;">
       <i style="background:green; width:15px; height:15px; display:inline-block; border-radius:50%; border:1px solid #666; vertical-align:middle;"></i> 
       <b>고비용 (Cost)</b><br>
       <span style="font-size:12px; color:#555; margin-left: 20px;">지도상 수치 = 배송비 비율</span>
     </div>
     
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

# 파일 저장
output_file = 'strategic_dashboard_map_v2.html'
m.save(output_file)
print(f"✅ 시각화 완료! '{output_file}' 파일을 열어보세요.")

🚀 데이터 로드 및 처리 시작...
✅ 마스터 테이블 생성 완료
📊 지역별 전략 지표 계산 중...
🗺️ 3. Folium 지도 생성 중...
✅ 시각화 완료! 'strategic_dashboard_map_v2.html' 파일을 열어보세요.
